# 3. Парные сравнения числовых данных

**Биномиальный тест**

* Тип: непараметрический.
* Применяется: предположение о том, **как часто должно случаться событие**, и мы хотим его проверить.
* Данные: переменная, которая принимает два значения.
* Нулевая гипотеза: нет разницы между фактическим и ожидаемым распределением.

>binomtest(bins[0], bins[1])

**T-test**

* Тип: параметрический
* Применяется: когда нужно понять, есть ли различия в среднем значении переменной между двумя группами наблюдений.
* Данные: должны состоять из двух переменных — по одной переменной они делятся на две группы, по другой — сравниваются между собой
* Нулевая гипотеза: средние значения переменной в двух группах не отличаются значимо

Используется в задачах:
- Cравнение средних в независимых группах	
- Cравнение средних в связанных группах
- Cравнение среднего одной группы с фиксированным показателем

Прежде чем проводить T-тест, необходимо проверить, что переменная для оценки:
+ непрерывна; 
+ распределена близко к нормальному (**критерий Шапиро — Уилка: shapiro(books)**);
+ обладает незначительными различиями в дисперсии между двумя группами наблюдений (**критерий Левене: levene(books_1, books_2)**).

> ttest_ind(books_1, books_2)




**ANOVA (ANalysis Of VAriance — «анализ дисперсии средних оценок»)**

* Тип: параметрический.
* Применяется: когда нужно понять, есть ли различия в среднем значении какой-то переменной, когда групп наблюдений больше чем две.
* Данные: должны состоять из двух переменных — по одной переменной они делятся на несколько групп, по другой сравниваются между собой.
* Нулевая гипотеза: нет различий в дисперсии между переменной внутри групп и между группами.

> f_oneway(books_1, books_2, books_3, books_4)

Типы ANOVA различаются по количеству независимых переменных:
* однофакторный дисперсионный анализ (с одной независимой переменной)
* двухфакторный дисперсионный анализ (с двумя независимыми переменными)

Набор данных «Amazon Top 50 Bestselling Books 2009 - 2019» состоит из информации о 50 самых популярных книг на Amazon за каждый год с 2009-го по 2019-й.

In [1]:
import pandas as pd
df = pd.read_csv('data/bestsellers.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [2]:
from scipy.stats import binom_test

# H0: «Вероятность встретить художественную и нехудожественную книгу в нашей подборке равна».

bins = df['Genre'].value_counts().tolist()
print(bins)

p_value = binom_test(bins)
print(f'p-value = {p_value:.5f}')

print('Принимаем') if p_value > 0.05 else print('Отвергаем нулевую гипотезу')

[310, 240]
p-value = 0.00322
Отвергаем нулевую гипотезу


In [3]:
# Проверка равенства средних значений в двух выборках (t-тест)

# H0: Между средними оценками художественных и нехудожественных книг нет значимого отличия.

# 1. Нормальность распределения средних в выборке. 
from scipy.stats import shapiro
H0 = 'Жанр Non Fiction распределен случанйным образом'
means_non_fic = []
for _ in range(500): # Cлучайным образом берем 100 случайных нехудожественных книг и вычислять их среднюю оценку. 
    sample_mean = df[df['Genre'] == "Non Fiction"].sample(100)['User Rating'].mean()
    means_non_fic.append(sample_mean)
shapiro_pvalue_non_fic = shapiro(means_non_fic).pvalue
print(f'Shapiro test for Non Fiction, p-value = {shapiro_pvalue_non_fic:.3f}')
print(f'Принимаем Н0: {H0}') if shapiro_pvalue_non_fic > 0.05 else print('Отвергаем Н0: {H0}')

H0 = 'Жанр Fiction распределен случанйным образом'
means_fic = []
for _ in range(500):
    sample_mean = df[df['Genre'] == "Fiction"].sample(100)['User Rating'].mean()
    means_fic.append(sample_mean)
shapiro_pvalue_fic = shapiro(means_fic).pvalue
print(f'\nShapiro test for Fiction, p-value = {shapiro_pvalue_fic:.3f}')
print(f'Принимаем Н0: {H0}') if shapiro_pvalue_fic > 0.05 else print('Отвергаем Н0: {H0}')


# 2. Равнозначность дисперсии по группам
from scipy.stats import levene
H0 = 'Различия дисперсии между группами не значимы'
user_rating_fic = df[df['Genre'] == "Fiction"].sample(100)['User Rating']
user_rating_non = df[df['Genre'] == "Non Fiction"].sample(100)['User Rating']
levene_pvalue = levene(user_rating_fic, user_rating_non).pvalue
print(f'\nLevene test, p-value = {levene_pvalue:.3f}')
print(f'Принимаем Н0: {H0}') if levene_pvalue > 0.05 else print('Отвергаем Н0: {H0}')


# 3. t-test
from scipy.stats import ttest_ind
H0 = 'Между средними оценками художественных и нехудожественных книг нет значимого отличия'
ttest_ind_pvalue = ttest_ind(user_rating_fic, user_rating_non, equal_var=True).pvalue # equal_var - применяет поправку Уэлча
print(f'\nT-test, p-value = {ttest_ind_pvalue:.3f}')
print(f'Принимаем Н0: {H0}') if ttest_ind_pvalue > 0.05 else print('Не можем принять нулевую гипотезу. Вывод: жанр связан с оценкой книг.')

from statistics import mean
print(f'\n{mean(user_rating_fic)}, {mean(user_rating_non)}')

Shapiro test for Non Fiction, p-value = 0.324
Принимаем Н0: Жанр Non Fiction распределен случанйным образом

Shapiro test for Fiction, p-value = 0.401
Принимаем Н0: Жанр Fiction распределен случанйным образом

Levene test, p-value = 0.134
Принимаем Н0: Различия дисперсии между группами не значимы

T-test, p-value = 0.119
Принимаем Н0: Между средними оценками художественных и нехудожественных книг нет значимого отличия

4.66, 4.608


Это может быть связано с тем, что оценка художественных книг зависит от того, нравятся они или не нравятся, 
а нехудожественных — от того, воспринимаются ли они как полезные или нет.

In [4]:
# Отличаются ли средние оценки книг за три последних года? 
# H0: «За последние три года книги в среднем оценивали одинаково».

from scipy.stats import f_oneway

ur_2019 = df[(df['Year'] == 2019)]['User Rating']
ur_2018 = df[(df['Year'] == 2018)]['User Rating']
ur_2017 = df[(df['Year'] == 2017)]['User Rating']

f_oneway_pvalue = f_oneway(ur_2019, ur_2018, ur_2017).pvalue
print(f'Принимаем Н0: Оценки в топе за год между собой не различаются') if f_oneway_pvalue > 0.05 else print('Не можем принять нулевую гипотезу.')



Принимаем Н0: Оценки в топе за год между собой не различаются


In [5]:
import pandas as pd
df = pd.read_csv('data/books_prep.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre,User Rating (Round),Price (Above Average)
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction,5,No
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction,5,Yes
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction,5,Yes
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction,5,No
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction,5,No


In [6]:
df['User Rating'].describe() # ['mean']

count    550.000000
mean       4.618364
std        0.226980
min        3.300000
25%        4.500000
50%        4.700000
75%        4.800000
max        4.900000
Name: User Rating, dtype: float64

In [7]:
# Выведите имя самого популярного писателя в наборе данных
name = df['Author'].describe()['top']
print(f'Имя самого популярного писателя - {name}')

mean_yes = df['User Rating'][df['Price (Above Average)'] == 'Yes'].describe()['mean']
mean_no = df['User Rating'][df['Price (Above Average)'] == 'No'].describe()['mean']
print(f'Cредние значения рейтинга для «дешёвых» книг {mean_no:.3f} и «дорогих» книг {mean_yes:.3f}')

user_rating_PriceYes = df['User Rating'][df['Price (Above Average)'] == 'Yes'] # .sample(100)['User Rating']
user_rating_PriceNo = df['User Rating'][df['Price (Above Average)'] == 'No'] # .sample(100)['User Rating']
# user_rating_PriceYes = df[df['Price (Above Average)'] == 'Yes'].sample(100)['User Rating']
# user_rating_PriceNo  = df[df['Price (Above Average)'] == 'No'].sample(100)['User Rating']
 
from scipy.stats import levene
levene_pvalue = levene(user_rating_PriceYes, user_rating_PriceNo).pvalue
print(f'Test Levene: p-value = {levene_pvalue:.3f}')

from scipy.stats import ttest_ind
ttest_ind_pvalue = ttest_ind(user_rating_PriceYes, user_rating_PriceNo).pvalue 
print(f'T-test, p-value = {ttest_ind_pvalue:.5f}')

from scipy.stats import f_oneway
# H0: «Рейтинг книги для каждой из групп не зависит от числа рецензий»
# (на хорошие книги люди охотнее пишут рецензии)
ur_3 = df[(df['User Rating (Round)'] == 3)]['Reviews']
ur_4 = df[(df['User Rating (Round)'] == 4)]['Reviews']
ur_5 = df[(df['User Rating (Round)'] == 5)]['Reviews']

f_oneway_pvalue = f_oneway(ur_3, ur_4, ur_5).pvalue
print(f'ANOVA Test: p-value = {f_oneway_pvalue:.3f}')

Имя самого популярного писателя - Jeff Kinney
Cредние значения рейтинга для «дешёвых» книг 4.650 и «дорогих» книг 4.560
Test Levene: p-value = 0.104
T-test, p-value = 0.00001
ANOVA Test: p-value = 0.297
